#  Personal Finance Management Application

# Database Initialization

In [1]:
import sqlite3
import pandas as pd

# Database Initialization
def init_db():
    conn = sqlite3.connect("finance.db")
    cursor = conn.cursor()
    # User table
    cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT UNIQUE NOT NULL,
                        password TEXT NOT NULL)''')
    # Transactions table
    cursor.execute('''CREATE TABLE IF NOT EXISTS transactions (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        user_id INTEGER,
                        type TEXT, 
                        category TEXT,
                        amount REAL,
                        date TEXT,
                        FOREIGN KEY(user_id) REFERENCES users(id))''')
    conn.commit()
    conn.close()
    print("Database Initialized.")

init_db()

Database Initialized.


# User Registration and Authentication

In [2]:
# User Registration
def register_user(username, password):
    conn = sqlite3.connect("finance.db")
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
        conn.commit()
        print(f"User '{username}' registered successfully!")
    except sqlite3.IntegrityError:
        print("Username already exists. Try a different one.")
    conn.close()

# User Authentication
def authenticate_user(username, password):
    conn = sqlite3.connect("finance.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE username = ? AND password = ?", (username, password))
    user = cursor.fetchone()
    conn.close()
    return user

# Example Execution
register_user("test_user", "password123")
print(authenticate_user("test_user", "password123"))

User 'test_user' registered successfully!
(1, 'test_user', 'password123')


# Income and Expense Tracking

In [3]:
from datetime import datetime

# Add Transaction
def add_transaction(user_id, t_type, category, amount):
    conn = sqlite3.connect("finance.db")
    cursor = conn.cursor()
    date = datetime.now().strftime("%Y-%m-%d")
    cursor.execute("INSERT INTO transactions (user_id, type, category, amount, date) VALUES (?, ?, ?, ?, ?)",
                   (user_id, t_type, category, amount, date))
    conn.commit()
    conn.close()
    print(f"{t_type.capitalize()} transaction added successfully!")

# View Transactions
def view_transactions(user_id):
    conn = sqlite3.connect("finance.db")
    df = pd.read_sql_query("SELECT * FROM transactions WHERE user_id = ?", conn, params=(user_id,))
    conn.close()
    return df

# Example Execution
user = authenticate_user("test_user", "password123")
if user:
    add_transaction(user[0], "income", "Salary", 5000)
    add_transaction(user[0], "expense", "Food", 50)
    print(view_transactions(user[0]))

Income transaction added successfully!
Expense transaction added successfully!
   id  user_id     type category  amount        date
0   1        1   income   Salary  5000.0  2024-12-14
1   2        1  expense     Food    50.0  2024-12-14


# Financial Reports

In [4]:
# Financial Report
def generate_report(user_id, period="monthly"):
    conn = sqlite3.connect("finance.db")
    if period == "monthly":
        query = "SELECT type, SUM(amount) AS total FROM transactions WHERE user_id = ? AND strftime('%Y-%m', date) = ? GROUP BY type"
        date_period = datetime.now().strftime("%Y-%m")
    elif period == "yearly":
        query = "SELECT type, SUM(amount) AS total FROM transactions WHERE user_id = ? AND strftime('%Y', date) = ? GROUP BY type"
        date_period = datetime.now().strftime("%Y")
    else:
        return "Invalid period. Use 'monthly' or 'yearly'."
    
    df = pd.read_sql_query(query, conn, params=(user_id, date_period))
    conn.close()
    print(f"--- {period.capitalize()} Report ---")
    return df

# Example Execution
if user:
    print(generate_report(user[0], period="monthly"))

--- Monthly Report ---
      type   total
0  expense    50.0
1   income  5000.0


# Budgeting

In [5]:
# Budgeting
def check_budget(user_id, category, budget_limit):
    conn = sqlite3.connect("finance.db")
    query = '''SELECT SUM(amount) AS total_expense FROM transactions
               WHERE user_id = ? AND type = 'expense' AND category = ?'''
    cursor = conn.cursor()
    cursor.execute(query, (user_id, category))
    result = cursor.fetchone()[0] or 0
    conn.close()
    if result > budget_limit:
        print(f"Alert! You've exceeded your budget for {category}. Spent: {result}, Limit: {budget_limit}")
    else:
        print(f"Budget check passed. Spent: {result}, Limit: {budget_limit}")

# Example Execution
if user:
    check_budget(user[0], "Food", 100)

Budget check passed. Spent: 50.0, Limit: 100


#  Full Flow Example

In [6]:
# Simulated User Flow
username = "test_user"
password = "password123"

# Register and Authenticate
register_user(username, password)
user = authenticate_user(username, password)

if user:
    user_id = user[0]
    add_transaction(user_id, "income", "Salary", 5000)
    add_transaction(user_id, "expense", "Food", 120)
    add_transaction(user_id, "expense", "Rent", 1500)
    print(view_transactions(user_id))
    print(generate_report(user_id, period="monthly"))
    check_budget(user_id, "Food", 100)
else:
    print("Authentication failed.")

Username already exists. Try a different one.
Income transaction added successfully!
Expense transaction added successfully!
Expense transaction added successfully!
   id  user_id     type category  amount        date
0   1        1   income   Salary  5000.0  2024-12-14
1   2        1  expense     Food    50.0  2024-12-14
2   3        1   income   Salary  5000.0  2024-12-14
3   4        1  expense     Food   120.0  2024-12-14
4   5        1  expense     Rent  1500.0  2024-12-14
--- Monthly Report ---
      type    total
0  expense   1670.0
1   income  10000.0
Alert! You've exceeded your budget for Food. Spent: 170.0, Limit: 100


# Testing

In [9]:
import unittest

class TestFinanceApp(unittest.TestCase):
    def setUp(self):
        """Initialize database and register a test user."""
        self.test_username = "unit_test_user"
        self.test_password = "test123"
        print("\nSetting up test user...")
        register_user(self.test_username, self.test_password)

    def test_register_user(self):
        """Test user registration."""
        print("Testing user registration...")
        user = authenticate_user(self.test_username, self.test_password)
        self.assertIsNotNone(user, "User registration failed.")
        print("User registration passed!")

    def test_authenticate_user(self):
        """Test user authentication."""
        print("Testing user authentication...")
        user = authenticate_user(self.test_username, self.test_password)
        self.assertIsNotNone(user, "User authentication failed.")
        print("User authentication passed!")

if __name__ == "__main__":
    unittest.main(argv=['', '-v'], exit=False)

test_authenticate_user (__main__.TestFinanceApp.test_authenticate_user)
Test user authentication. ... ok
test_register_user (__main__.TestFinanceApp.test_register_user)
Test user registration. ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.007s

OK



Setting up test user...
Username already exists. Try a different one.
Testing user authentication...
User authentication passed!

Setting up test user...
Username already exists. Try a different one.
Testing user registration...
User registration passed!
